In [13]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [14]:
import copy
import datetime
import os
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [15]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
command, segment, block, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)
sar = celeri.process_sar(sar, command)
segment, station, block, mogi, sar = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = celeri.create_assembly_dictionary()
assembly = celeri.merge_geodetic_data(assembly, station, sar) 

In [16]:
# celeri.plot_block_labels(segment, block, station)

In [17]:
def get_block_constraint_partials(block):
    """
    Partials for a priori block motion constraints.
    Essentially a set of eye(3) matrices
    """
    n_blocks = len(block)
    apriori_blocks_idx = np.where(block.apriori_flag.to_numpy() == 1)
    partials = np.zeros(3 * len(apriori_blocks_idx), 3 * n_blocks)
    for i in range(len(apriori_blocks_idx)):
    #     partials(3*(i-1) + (1:3), 3*(ap(i)-1) + (1:3)) = np.eye(3)
    return partials

def block_constraints(block, index, data, sigma, command):
    """
    Applying a priori block motion constraints
    """
    block_constraint_partials = get_block_constraint_partials(block)
    index.block_constraints = np.where(block.apriori_flag.to_numpy == 1)
    data.n_block_constraints = 3 * len(index.block_constraints)
    data.block_constraints = np.zeros(block_constraint_partials.size[0])
    sigma.block_constraints = np.zeros(block_constraint_partials.size[0])
    if data.n_block_constraints > 0:
    #    apLons, apLats, apRates = deal(Block.eulerLon(Index.blockCon), Block.eulerLat(Index.blockCon), Block.rotationRate(Index.blockCon).*1e6)
    #    apbx, apby, apbz = EulerToOmega(apLons, apLats, apRates)
    #    data.block_constraints[0::3] = apbx
    #    data.block_constraints[1::3] = apby
    #    data.block_constraints[2::3] = apbz
    #    apbcov = stack3([deg_to_rad(Block.eulerLatSig(Index.blockCon)), deg_to_rad(Block.eulerLonSig(Index.blockCon)), deg_to_rad(Block.rotationRateSig(Index.blockCon))])
    #    apbsx, apbsy, apbsz = epoles_cov_to_omega_cov(apbx, apby, apbz, diag(apbcov))
    #    sigma.block_constraints[0::3] = apbsx
    #    sigma.block_constraints[1::3] = apbsy
    #    sigma.block_constraints[2::3] = apbsz

    sigma.block_constraint_weight = command["block_constraint_weight"]
    return block_constraint_partials, index, data, sigma

IndentationError: expected an indented block (<ipython-input-17-866dc5966786>, line 11)

In [ ]:
a = {}
a["asdf"] = 1
a["er"] = {}
a["er"]["asdf"] = 2 